In [29]:
import keras

import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [2]:
from music21 import converter, note, chord, instrument

In [3]:
notes = []
file = "music_src/*"
midi = converter.parse(file)
notes_to_parse = None

parts = instrument.partitionByInstrument(midi)

print(parts)

if parts: # file has instrument parts
    notes_to_parse = parts.parts[0].recurse()
else: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

print(notes_to_parse)

for element in notes_to_parse:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))
        #
        
print(notes)

n_vocab = len(set(notes))



<music21.stream.Score 0x13c40bdd8>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
['A5', 'A3', 'E6', 'E4', 'D6', 'D4', 'E6', 'E4', 'D6', 'D4', 'C6', 'C4', 'D6', 'D4', 'E6', 'E4', 'F6', 'F4', 'G6', 'G4', 'E6', 'E4', 'F6', 'F4', 'E6', 'E4', 'D6', 'E6', 'D4', 'E4', 'D6', 'D4', 'C6', 'C4', 'B-5', 'B-3', 'A2', 'A3', 'E3', 'C#4', 'A3', 'E4', 'C#4', 'A4', 'E4', 'C#5', 'A4', 'E5', 'C#5', 'A4', 'C#4', 'C#5', 'E4', 'E5', 'A4', 'A5', 'C#5', 'C#6', 'E5', 'E6', 'A5', 'C#6', 'E5', 'A5', 'C#5', 'E5', 'A4', 'C#5', 'E4', 'A4', 'C#4', 'E5', 'A4', 'C#5', 'E4', 'A4', 'C#4', 'E4', 'A3', 'C#4', 'A3', 'A3', 'E3', 'A2', 'A5', 'A3', 'E6', 'E4', 'D6', 'D4', 'E6', 'E4', 'D6', 'D4', 'C6', 'C4', 'D6', 'D4', 'E6', 'E4', 'F6', 'F4', 'G6', 'G4', 'E6', 'E4', 'F6', 'F4', 'E6', 'E4', 'D6', 'E6', 'D4', 'E4', 'D6', 'D4', 'C6', 'C4', 'B-5', 'B-3', 'A5', 'A3', 'B-5', 'B-3', 'C6', 'C4', 'D6', 'D4', 'F6', 'F4', 'D6', 'E6', 'D4', 'E4', 'D6', 'D4', 'C6', 'C4', 'B-5', 'B-3', 'A5', 'A3', 'B-5', 'B-3', 'C6', 'C4', 

In [4]:
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [5]:
model = Sequential()
model.add(LSTM(256,
               input_shape=(network_input.shape[1], network_input.shape[2]),
               return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [6]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=20, batch_size=64, callbacks=callbacks_list)

Epoch 1/20


 64/522 [==>...........................] - ETA: 33s - loss: 3.8293

128/522 [======>.......................] - ETA: 20s - loss: 3.7842

192/522 [==========>...................] - ETA: 16s - loss: 4.0134

256/522 [=============>................] - ETA: 12s - loss: 4.0008

...

320/522 [=================>............] - ETA: 6s - loss: 3.2962

384/522 [=====================>........] - ETA: 4s - loss: 3.3475

448/522 [========================>.....] - ETA: 2s - loss: 3.3407

512/522 [============================>.] - ETA: 0s - loss: 3.3172

522/522 [==============================] - 15s 30ms/step - loss: 3.3151


In [10]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('weights-improvement-19-3.2901-bigger.hdf5')

In [26]:
start = numpy.random.randint(0, len(network_input) -1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    #print(prediction_input)
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    numpy.append(pattern,index)
    #pattern.append(index)
    pattern = pattern[0:len(pattern)]

In [27]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [30]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'